# Calcualte the actual Perturbation Interactions  
1.) Extract all perturbation vectors

Perfore calculating the actual interaction one need to consider:  
2.) Check Drug Decay  (some drugs show symptoms of decay e.g. oxidiation --> less potency over time)  
3.) Check CellCount  (if ther cellcount is below a certain number not statistical significant assumption can be made regarding the cell morphology)  

To calculate meaningful interaction one need to check if the single and combination perturbation are significantly different from DMSO random fluctuation  
4.) Check significance of single drugs   

Finally drug interaction can be calculated  
5.) Check per Single Drug fluctuation (alpha/beta and gamma) 



In [ ]:
# import all necessary python modules
import pickle
import numpy as np
import random
from sklearn.decomposition import PCA
from sklearn.neighbors.kde import KernelDensity

from scipy.spatial import distance as dis
from scipy.spatial.distance import mahalanobis
from matplotlib import pylab as plt
from matplotlib.patches import Patch

import seaborn as sns

import os

## 1. Extract perturbation vectors
Load both the n-dim perturbation vectors for cells being (i) untreated and (ii) treated with any single or combination of drugs

In [ ]:
'''
Extract the perturbation vectors as well as DMSO vectors (split per batch to reduce per batch effects)
a.) drug_perturbation_vectors (drug perturbations)
b.) dmso (DMSO)
'''


# Load all the drug perturbation vectors
path = '../data/Calculate_Interactions/All_Vectors_Combined.csv'
fp = open(path)

# Get a list of all features
features = fp.readline().split(',')[1:]
numfeatures = len(features)

# Final dictionaries containing the individual perturbation vectors
drug_perturbation_vectors = {'Batch1':{},'Batch2':{}}
dmso = {'Batch1':[],'Batch2':[]}

All_CLOUDs = set()
# Go threw the file and create DMSO and drug_perturbation_vectors
# DMSO per batch
# drug_perturbation_vectors per Batch with full identifier
for line in fp:
    tmp = line.strip().split(',')
    
    drug1, drug2 = tmp[0].split('_')[0].split('|')
    well = tmp[0].split('_')[1]
    plate = tmp[0].split('_')[2]
    
    values = list(np.float_(tmp[1:]))
    
    if int(plate) < 1315065:
        if drug1 == 'DMSO':
            dmso['Batch1'].append(values)
        drug_perturbation_vectors['Batch1'][drug1+','+drug2+','+plate+','+well] = values

            
    else:
        if drug1 == 'DMSO':
            dmso['Batch2'].append(values)
        drug_perturbation_vectors['Batch2'][drug1+','+drug2+','+plate+','+well] = values
        
        #If at least in at least any of either two batches
        All_CLOUDs.add(drug1)

        
all_all_clouds = []
for i in range(1,268):
    all_all_clouds.append('CLOUD'+str(i).zfill(3))
    
#Get list of All_CLOUDs, this number is smaller than the original number due to some drugs not being transformer correctly or other problems
All_CLOUDs.remove('DMSO')
All_CLOUDs.remove('PosCon')

#plot number of clouds found
All_CLOUDs = list(All_CLOUDs)
All_CLOUDs.sort()
print 'Number of drugs that have at least one correct well: %d' %len(All_CLOUDs)
    
print len(drug_perturbation_vectors['Batch2'][drug_perturbation_vectors['Batch2'].keys()[0]])

## 2. Check Drug Decay
Check if some of the drugs decayed over the course of the screen. This can be validated by checking if drugs have a different cytotoxicity between 'early' spotted plates compared to 'later' spotted plates. Plates with lower plate IDs are earlier.

In [ ]:
# Both thresholds need to be true to set a drug as decayed during experiment; threshold_decay is steepness and threshold_MaxDifference absolute difference
threshold_decay = 0.05
threshold_MaxDifference = 0.3


# Load all the drug decay regressions
# Created by checking the single drug responses over the different plates (there is a temporal context between plate 1 and 123)
# One is interested both in the decay as well as the maximum change e.g. if gradient between 0.1 to 0.2, still ok
# Create a dic that tells about the status of drug decay i.e. True if drug WORKED CORRECTLY
path = '../data/Calculate_Interactions/DrugDecay_Combined.csv'
fp = open(path)
fp.next()
drug_decay = {}
batch1_Failed = 0
batch2_Failed = 0
for line in fp:
    tmp = line.strip().split(',')
    
    batch1_decay = float(tmp[1])
    batch1_diff = float(tmp[2])
    
    batch2_decay = float(tmp[3])
    batch2_diff = float(tmp[4])
    
    
    batch1_Status = True
    if batch1_decay >= threshold_decay and batch1_diff >= threshold_MaxDifference:
        batch1_Status = False
        batch1_Failed += 1
        
    batch2_Status = True
    if batch2_decay >= threshold_decay and batch2_diff >= threshold_MaxDifference:
        batch2_Status = False
        batch2_Failed += 1
    
    
    drug_decay[tmp[0]] = {'Batch1':batch1_Status,'Batch2':batch2_Status}
fp.close()

print 'Number of drugs that decayed in batch1: %d' %batch1_Failed
print 'Number of drugs that decayed in batch2: %d' %batch2_Failed

## 3. Check Cell Count
Check if the individual wells have enough cells to be statistically valid. Having wells with only e.g. 5 cells can all be outliers or somehow else be errourous

In [ ]:
#Minimum cells allowed
cutoff_min_cells = 30

#per well count
well_cell_count = {}

#number of wells below cutoff_min_cells
empty_well = 0

#go through CellCount file to find wells with too little cells
path = '../data/Calculate_Interactions/All_CellCounts_Combined.csv'
fp = open(path)
fp.next()
for line in fp:
    tmp = line.strip().split(',')
    num_cells = float(tmp[4])
    well_cell_count[tmp[0]+','+tmp[1]+','+tmp[2]+','+tmp[3]] = {'Number':num_cells,'Worked':tmp[5]}
    
#Mean number of cells (DMSO and Perturbations)
print 'Mean number of cells in Batch1: %f'  %np.mean([well_cell_count[x]['Number'] for x in well_cell_count if int(x.split(',')[2]) < 1315065])
print 'Mean number of cells in Batch1: %f'  %np.mean([well_cell_count[x]['Number'] for x in well_cell_count if int(x.split(',')[2]) >= 1315065])

#Number of empty wells (DMSO and Perturbations)
print 'Number of empty wells in Batch1: %d'  %len([well_cell_count[x]['Number'] for x in well_cell_count if int(x.split(',')[2]) < 1315065 and well_cell_count[x]['Number'] < cutoff_min_cells])
print 'Number of empty wells in Batch2: %d'  %len([well_cell_count[x]['Number'] for x in well_cell_count if int(x.split(',')[2]) >= 1315065 and well_cell_count[x]['Number'] < cutoff_min_cells])

#Get the 90 percentile of DMSO cellcount
DMSO_CellCount = {}
DMSO_CellCount['Batch1']= np.percentile([well_cell_count[x]['Number'] for x in well_cell_count if int(x.split(',')[2]) < 1315065 and x.split(',')[0] == 'DMSO'],90)
DMSO_CellCount['Batch2']= np.percentile([well_cell_count[x]['Number'] for x in well_cell_count if int(x.split(',')[2]) >= 1315065 and x.split(',')[0] == 'DMSO'],90)

### Check which Drugs should be removed from analysis  (too few cells, transfer problems etc.)

In [ ]:
#list of drugs that kill too many cells to do morphological analysis with them (Remove only for Morphology Analysis)
killing_drugs = {'Batch1':[],'Batch2':[]}

#this drugs must be removed from further analysis (Both the CellCount and Morphology Analysis)
drugs_to_remove = {'Batch1':[],'Batch2':[]}

for cloud in All_CLOUDs:
    for b in ['Batch1','Batch2']:
        
        #Check if enough replicates exist
        enough_replicates_status = len(drug_vectors_WithoutOutliers[b][cloud]) >= 3
        
        #Get result from drug decay analayis
        drug_decay_status = drug_decay[cloud][b]
        
        #Check if both are good
        if enough_replicates_status and drug_decay_status:   
            continue
        else:
          
            #if the drug does not decay and has enough working wells not image problems, you can use it still for CellCount Anlaysis (again if at least 3 wells)
            if drug_decay_status:
                number_replicates = len([x for x in drug_perturbation_vectors[b] if cloud+',DMSO' in x])
                
                if number_replicates >= 3:
                    #print '==> can be used a "killing drug"'
                    killing_drugs[b].append(cloud)
                else:
                    drugs_to_remove[b].append(cloud)
            else:
                drugs_to_remove[b].append(cloud)

print 'Drugs to remove:'
print drugs_to_remove

print 'Cytotoxic drugs/non usable morphological features:'
print killing_drugs

## 4. Check which single Drugs are significantly morphologically changed
Compared to DMSO random cell intrinsic morphological fluctuation (compare via Mahalanobis distance)

In [ ]:
# Some Easy Outlier detection
def reject_outliers_2(data, m=6.):
    d = np.abs(data - np.median(data))
    mdev = np.median(d)
    s = d / (mdev if mdev else 1.)
    return s < m
    #return [data[i] for i in range(0, len(data)) if s[i] < m]

    

def Create_Single_Drug_Vectors():
    '''
    Extracts the individual drug perturbation vectors (for the two batches separate).
    Outliers e.g. one replicate that is too different from the remaining 5/6 will be removed.
    
    Only calculated for significant singles
    '''
    
    drug_vectors = {'Batch1':{},'Batch2':{}}
    for cloud in All_CLOUDs:
        for b in ['Batch1','Batch2']:    


            drug_wells = [x for x in drug_perturbation_vectors[b] if cloud+',DMSO' in x and well_cell_count[x]['Number'] > cutoff_min_cells]

           
            if len(drug_wells) < 3:
                drug_vectors[b][cloud] = {}
            else:
                drug_well_values = []
                for well in drug_wells:
                    drug_well_values.append(drug_perturbation_vectors[b][well])

                drug_well_values = np.array(drug_well_values)

                drug_EucledianDistances = []
                drug_names = []
                drug_values = []
                for s1_a,label_a in zip(drug_well_values,drug_wells):
                    tmp = []
                    for s1_b,label_b in zip(drug_well_values,drug_wells):
                        sim = np.linalg.norm(s1_a - s1_b) 
                        tmp.append(sim)

                    drug_EucledianDistances.append(np.mean(tmp))
                    drug_names.append(label_a)
                    drug_values.append(s1_a)

                
               
                good_rows = reject_outliers_2(drug_EucledianDistances)
                keep_values = [drug_values[x] for x in range(0,len(drug_values)) if good_rows[x]]
                keep_names = [drug_names[x] for x in range(0,len(drug_names)) if good_rows[x]]

                drug_vectors[b][cloud] = {}
                for val, lab in zip(keep_values,keep_names):
                    drug_vectors[b][cloud][lab] = val

    return drug_vectors


In [ ]:
'''
Create a dictionary for all single drugs, and remove obvious outliers
Outliers are detected by first calculating the mean eucledian distance of a point to all other points and then performing an MAD outlier detection
'''
drug_vectors_WithoutOutliers = Create_Single_Drug_Vectors()

In [ ]:
def make_Random_Distribution_DMSO(DMSO_Wells,num_pseudo_treatments=4):
    '''
    Calculate random mahalanbis distances between randomly picked DMSO wells. Computing these values beforehand significantly improves performance, else for each significance calculation
    this random distribution would be need to be calculated.
    '''
    
    
    #Fit a PCA
    pca = PCA()
    pca.fit(DMSO_Wells)

    #Take only as many principel components so taht 90% of the variance can be explained
    variances = pca.explained_variance_ratio_
    total = 0
    use_components = 0
    for v in variances:
        total = total + v
        use_components += 1
        if total > 0.9:
            break

    #Use at least 2 components
    if use_components <= 1:
        use_components = 2
    
    #Fit the space to the selected amount of components and transform the data
    pca = PCA(n_components=use_components)
    pca.fit(DMSO_Wells)
    transformedX = pca.transform(DMSO_Wells)
    weightedPCA = np.multiply(transformedX, pca.explained_variance_ratio_)
    
    all_mahala_distances = []
    for i in range(0, 10000):
        np.random.shuffle(weightedPCA)

        treatments = weightedPCA[0:num_pseudo_treatments]
        control = weightedPCA[num_pseudo_treatments:]

        x = []
        for i in range(0, use_components):
            x.append(np.mean(treatments[:, i]))

        u = []
        for i in range(0, use_components):
            u.append(np.mean(control[:, i]))

        covariance_treatment = np.cov(treatments, rowvar=0)
        covariance_control = np.cov(control, rowvar=0)

        weighted_covariance_treatment = covariance_treatment * (float(len(treatments)) / (len(treatments) + len(control)))
        weighted_covariance_control = covariance_control * (float(len(control)) / (len(treatments) + len(control)))

        S = weighted_covariance_treatment + weighted_covariance_control

        S = np.cov(weightedPCA, rowvar=0)

        x = np.array(x)
        u = np.array(u)
        distance_rand = mahalanobis(x, u, np.linalg.inv(S))
        all_mahala_distances.append(distance_rand)
    
    
    return all_mahala_distances
    
#If the random DMSO distances were already calculated load pre-calculated distances
if os.path.isfile('../results/Calculate_Interactions/RandomMahalanobisDistances_DMSO.pickle'):
    pickle_in = open('../results/Calculate_Interactions/RandomMahalanobisDistances_DMSO.pickle',"rb")
    Mahalanobis_Random_Distribution = pickle.load(pickle_in)
else:
    #
    # Since there are more than 20k points of DMSO, the theoretically added 3-6 single perturbation points would not add too much but cost massive additionaly computaion time
    
    #Create Random Mahalanobis distributions for k = 1, 3,4,5,6,7 (1 = combination, 3-7 singles)
    Mahalanobis_Random_Distribution = {'Batch1':{},'Batch2':{}}
    for i in range(3,8):
        Mahalanobis_Random_Distribution['Batch1'][i] = make_Random_Distribution_DMSO(dmso['Batch1'],i)
    Mahalanobis_Random_Distribution['Batch1'][1] = make_Random_Distribution_DMSO(dmso['Batch1'],1)
    print 'Finished Batch1 Random Distribution'
    for i in range(3,8):
        Mahalanobis_Random_Distribution['Batch2'][i] = make_Random_Distribution_DMSO(dmso['Batch2'],i)
    Mahalanobis_Random_Distribution['Batch2'][1] = make_Random_Distribution_DMSO(dmso['Batch2'],1)
    print 'Finished Batch2 Random Distribution'

    #save the results as pickle
    pickle_out = open('../results/Calculate_Interactions/RandomMahalanobisDistances_DMSO.pickle',"wb")
    pickle.dump(Mahalanobis_Random_Distribution, pickle_out)
    pickle_out.close()



In [ ]:
def Calculate_MahalanobisDistance(k,numberTreatment,number_of_RandomTries=10000, usePreComputedRandomDistribution = True, batch = None):
    '''
    This function takes a list of feature vectors, sorted with the first n (numberTreatment) belong to group A and the afterwards to group B.
    1.) First the dimension of the feature vectors is reduced using PCA
    2.) Only the m PCA dimensions are used so taht at least 90% of all variance is explained
    3.) The indivudal dimensions are weighted by their importance
    4.) Create the Mean Vector for the two grups (mean over columns)
    5.) Calculate the Mahalanobis distance
    6.) Calculate an empirical PValue by randomly permuting the labels of the groups and creating a random expected mahalanobis distance distribution; compare real mahalanobis distance
        to this randomly drawn distribution to calulate a p value (basically: NumberOfRandomDrawn MahalanobisDistance larger than real / number of random Mahalanobis Distances)
    '''
    #Fit a PCA
    pca = PCA()
    pca.fit(k)

    #Take only as many principel components so taht 90% of the variance can be explained
    variances = pca.explained_variance_ratio_
    
    total = 0
    use_components = 0
    for v in variances:
        total = total + v
        use_components += 1
        if total > 0.9:
            break

    #Use at least 2 components
    if use_components <= 1:
        use_components = 2
    

    
    #Fit the space to the selected amount of components and transform the data
    pca = PCA(n_components=use_components)
    pca.fit(k)
    transformedX = pca.transform(k)

    #weight the dimensions regarding the amount of variance explained
    weightedPCA = np.multiply(transformedX, pca.explained_variance_ratio_)

    
    pca1 = list(weightedPCA[:, 0])
    pca2 = list(weightedPCA[:, 1])
    
    #split the list into the two groups (treated/untreated)
    treatments = weightedPCA[0:numberTreatment]
    control = weightedPCA[numberTreatment:]
    
    
    #Contains the main treatment vector (Calculate mean vector - Over Columns)
    x = []
    for i in range(0, use_components):
        x.append(np.mean(treatments[:, i]))

    #Contains the mean untreated vector (Calculate mean vector - Over Columns)
    u = []
    for i in range(0, use_components):
        u.append(np.mean(control[:, i]))

    
        
    #Create the covariance matrix, as well as the groupA (treatment) mean vector and
    # groupB (untreated) mean vector to finally calucate the mahalanobis distance between this two groups
    S = np.cov(weightedPCA, rowvar=0)
    x = np.array(x)
    u = np.array(u)

    
    #Calculate the Mahalanobis distance between treated and untreated
    distance_calc = mahalanobis(x, u, np.linalg.inv(S))

    #original = weightedPCA.copy()
    treatment_row = weightedPCA[0:numberTreatment].copy()

    #typically use pre computed distribution (marginally difference but much faster)
    if usePreComputedRandomDistribution == True:
        all_mahala_distances = Mahalanobis_Random_Distribution[batch][numberTreatment]
    else:
        
        #in case you stay with the at the time computation: calculate a random mahalanobis distance k times and compare with real mahalanobis distance
        
        all_mahala_distances = []
        for i in range(0, number_of_RandomTries):
            np.random.shuffle(weightedPCA)


            #if np.array_equal(original, weightedPCA):
            if np.array_equal(treatment_row,weightedPCA[0:numberTreatment]):
                continue

            treatments = weightedPCA[0:numberTreatment]
            control = weightedPCA[numberTreatment:]

            x = []
            for i in range(0, use_components):
                x.append(np.mean(treatments[:, i]))

            u = []
            for i in range(0, use_components):
                u.append(np.mean(control[:, i]))

                
            #Calculate random mahalanobis distance
            covariance_treatment = np.cov(treatments, rowvar=0)
            covariance_control = np.cov(control, rowvar=0)

            weighted_covariance_treatment = covariance_treatment * (float(len(treatments)) / (len(treatments) + len(control)))
            weighted_covariance_control = covariance_control * (float(len(control)) / (len(treatments) + len(control)))

            S = weighted_covariance_treatment + weighted_covariance_control

            S = np.cov(weightedPCA, rowvar=0)

            x = np.array(x)
            u = np.array(u)
            distance_rand = mahalanobis(x, u, np.linalg.inv(S))
            
            #add to all random
            all_mahala_distances.append(distance_rand)

    #caluclate empirical pvalue
    mp = len([x for x in all_mahala_distances if x >= distance_calc]) / float(len(all_mahala_distances))        
    if mp > 1:
        mp = 1

    
    return distance_calc, mp, pca1, pca2


In [ ]:
###
## LOAD SINGLES i.e. if not yet calculated, compute the significance (mahalanobis) values for each single drug treatment
###


print "Load singles:"

#set the perturbation significance level
perturbaion_significance = 7

#save signficance (MP_Value is deprecated)
Single_Drug_Significance = {}

#If this file already exists it doesn't need to be calculated again (if you wish to recalculate it you need to delete it first)
if os.path.isfile('../results/Calculate_Interactions/Singles/Overview.csv'):
    fp = open('../results/Calculate_Interactions/Singles/Overview.csv','r')
    fp.next()
    for line in fp:
        tmp = line.strip().split(',')
        Single_Drug_Significance[tmp[0]] = {'Batch1':{},'Batch2':{}}
        
        if tmp[1] == 'No_Cells':
            Single_Drug_Significance[tmp[0]]['Batch1']['Mahalanobis_Distance'] = 'Nan'
            Single_Drug_Significance[tmp[0]]['Batch1']['MP_Value'] = 'Nan'
        else:
            Single_Drug_Significance[tmp[0]]['Batch1']['Mahalanobis_Distance'] = float(tmp[1])
            Single_Drug_Significance[tmp[0]]['Batch1']['MP_Value'] = float(tmp[2])
            
        if tmp[3] == 'No_Cells':
            Single_Drug_Significance[tmp[0]]['Batch2']['Mahalanobis_Distance'] = 'Nan'
            Single_Drug_Significance[tmp[0]]['Batch2']['MP_Value'] = 'Nan'
        else:
            Single_Drug_Significance[tmp[0]]['Batch2']['Mahalanobis_Distance'] = float(tmp[3])
            Single_Drug_Significance[tmp[0]]['Batch2']['MP_Value'] = float(tmp[4])
            
else:
    #Go through all clouds
    for cloud in all_all_clouds:
        #cloud = 'CLOUD031'
        Single_Drug_Significance[cloud] = {}
        #calculate the significance for both batches separately
        for b in ['Batch1','Batch2']:

            #Create a list with Treatments (drug vectors) and DMSO; Including only those perturbations that have enough cells
            
            #Get the single perturbations
            
            if drug_vectors_WithoutOutliers[b].has_key(cloud):
                k = drug_vectors_WithoutOutliers[b][cloud].values()
                numberTreatment = len(k)
            else:
                numberTreatment = 0
            
            
            #Only perform an analysis if at least 3 replicates exist
            if numberTreatment < 3 or cloud in drugs_to_remove[b] or cloud in killing_drugs[b]:
                Single_Drug_Significance[cloud][b] = {'Mahalanobis_Distance':'LessThanThreeWells','MP_Value':'LessThanThreeWells'}
            elif cloud in killing_drugs[b]:
                Single_Drug_Significance[cloud][b] = {'Mahalanobis_Distance':'TooFewCells','MP_Value':'TooFewCells'}
            elif cloud in drugs_to_remove[b]:
                Single_Drug_Significance[cloud][b] = {'Mahalanobis_Distance':'DrugDecay','MP_Value':'DrugDecay'}
            else:
                #if enough replicates, add the DMSO wells to the list
                k.extend(dmso[b])

                #Calculate the mahalanobis distance as well as empirical pValue.
                distance_calc, mp, pca1, pca2 = Calculate_MahalanobisDistance(k,numberTreatment,usePreComputedRandomDistribution=True,batch=b)
                

                '''
                PLOT RESULTS
                '''
                if distance_calc >  perturbaion_significance:
                    color = ['#b2182b']*numberTreatment
                else:
                     color = ['#2166ac']*numberTreatment

                for i in range(1,len(k)):
                    color.append('grey')

                Single_Drug_Significance[cloud][b] = {'Mahalanobis_Distance':distance_calc,'MP_Value':mp}

                plt.scatter(pca1[numberTreatment:], pca2[numberTreatment:], c=color[numberTreatment:], alpha=0.4)
                plt.scatter(pca1[0:numberTreatment], pca2[0:numberTreatment], c=color[0:numberTreatment], alpha=0.4)
                plt.legend(['DMSO','Mahalanobis Distance: %.2f\nn = %d' % (distance_calc,numberTreatment)])
                #plt.show()
                plt.savefig('../results/Calculate_Interactions/Singles/'+cloud+'_'+b+'.pdf')
                plt.close()
    
    
    '''
    WRITE OUTPUT
    '''
    fp_out = open('../results/Calculate_Interactions/Singles/Overview.csv','w')
    fp_out.write('Drug,Batch1_Mahalanobis_Distance,Batch1_MP_Value,Batch2_Mahalanobis_Distance,Batch2_MP_Value\n')
    for cloud in all_all_clouds:
        fp_out.write(cloud+','+str(Single_Drug_Significance[cloud]['Batch1']['Mahalanobis_Distance'])+','+str(Single_Drug_Significance[cloud]['Batch1']['MP_Value'])+','+str(Single_Drug_Significance[cloud]['Batch2']['Mahalanobis_Distance'])+','+str(Single_Drug_Significance[cloud]['Batch2']['MP_Value'])+'\n')
    fp_out.close()


#print Single_Drug_Significance
print 'Number of significant drugs in Batch1: %d' %len([x for x in Single_Drug_Significance if Single_Drug_Significance[x]['Batch1']['Mahalanobis_Distance'] > perturbaion_significance and Single_Drug_Significance[x]['Batch2']['Mahalanobis_Distance']  != 'Nan'])
print 'Number of significant drugs in Batch2: %d' %len([x for x in Single_Drug_Significance if Single_Drug_Significance[x]['Batch2']['Mahalanobis_Distance'] > perturbaion_significance and Single_Drug_Significance[x]['Batch2']['Mahalanobis_Distance']  != 'Nan'])

In [ ]:
#Make a distribution over all perturbations; use mean over both batches
mahalanobis_distances = []
for cloud in All_CLOUDs:
    
    values = []
    if Single_Drug_Significance[cloud]['Batch1']['Mahalanobis_Distance'] != 'No_Cells' and Single_Drug_Significance[cloud]['Batch1']['Mahalanobis_Distance'] != 'Nan':
        values.append(Single_Drug_Significance[cloud]['Batch1']['Mahalanobis_Distance'])
    if Single_Drug_Significance[cloud]['Batch2']['Mahalanobis_Distance'] != 'No_Cells'  and Single_Drug_Significance[cloud]['Batch2']['Mahalanobis_Distance'] != 'Nan':
        values.append(Single_Drug_Significance[cloud]['Batch2']['Mahalanobis_Distance'])
    
    if len(values) > 0:
        mahalanobis_distances.append(max(values))


##
# PLOT OUTPUT
##
print 'Number at least once significant: %d' %len([x for x in mahalanobis_distances if x > 7])
print 'Number never  significant: %d' %len([x for x in mahalanobis_distances if x <= 7])
plt.hist(mahalanobis_distances,bins=15, color='#40B9D4')
plt.axvline(7, color='grey', ls='--')
#plt.show()
plt.savefig('../results/Calculate_Interactions/Singles/Overview_Histogram.pdf')
plt.close()

## 5. Calculate Interactions
Calculate the actual interaction between all drug pairs. Include only drug pairs where both singles as well as the combination fullfill all quality control criteria 

### Create a dictionary of DMSO wells after a (MAD=2) outlier detection; these will serve as random intracellular fluctuation

In [ ]:
#This dictonary will contain the 15-32 DMSO wells of each plate
dmso_wells_Outlier_Removed = {'Batch1':{},'Batch2':{}}

#Go through both batches
for b in ['Batch1','Batch2']:
    #get all plates of this batch
    plates = list(set([x.split(',')[2] for x in drug_perturbation_vectors[b].keys()]))
    #go through all plates
    for plate in plates:
        
        #Get corresponding DMSO wells of this plate
        DMSO_Wells  = [drug_perturbation_vectors[b][x] for x in drug_perturbation_vectors[b] if 'DMSO,None,'+str(plate) in x and well_cell_count[x]['Number'] > cutoff_min_cells]
        DMSO_Wells = np.array(DMSO_Wells)
        
        #Perform outlier detection
        DMSO_EucledianDistances = []
        DMSO_values = []
        for s1_a in DMSO_Wells:
            tmp = []
            for s1_b in DMSO_Wells:
                sim = np.linalg.norm(s1_a - s1_b) 
                tmp.append(sim)

            DMSO_EucledianDistances.append(np.mean(tmp))
            DMSO_values.append(s1_a)
        
        #Perform actual MAD outlier detection with MAD = 10
        good_rows = reject_outliers_2(DMSO_EucledianDistances,2)
        keep_values = [DMSO_values[x] for x in range(0,len(DMSO_values)) if good_rows[x]]
        
        #Add result
        dmso_wells_Outlier_Removed[b][plate] = keep_values
        

### Vector Math

In [ ]:
########################################
##Actual Math for calculating the DDIs##
########################################


def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)
def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966
            angle_between((1, 0, 0), (1, 0, 0))
            0.0
            angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))
def calculate_vector_math_v2(a, b, c):
    '''
    calculate the amount of single a, single b, and the 'surprise factor)
    :param a: vector a (single)
    :param b: vector b (single)
    :param c: vector c (combination)
    :return: alpha, beta and gamma (part of vector a, b and suprise)
    '''

    if sum(a) != 0 and sum(b) != 0:

        
        #Check first for two special cases:
        
        # 1.
        #Check if angle between the two vectors is to narrow (both point in same direction)
        if angle_between(a,b) <= 0.5:
            #h = c/(a+b)

            A=  np.array([[np.dot(a, a), np.dot(a, b)], [np.dot(a, b), np.dot(b, b)]])
            h = np.array([np.dot(a, c), np.dot(b, c)])

            alpha = h[0] / (A[0][0]+A[1][0])
            beta = h[1]  / (A[0][1]+A[1][1])

            n =alpha * a + beta * b - c

            gamma = np.linalg.norm(n)
            return str(alpha),str(beta),str(gamma)
        
        # 2. 
        #Check if angle between two is EXACTLY 180 degrees
        elif angle_between(a,b) == 3.141592653589793:
            A = np.array([[np.dot(a, a), np.dot(a, b)], [np.dot(a, b), np.dot(b, b)]])
            h = np.array([np.dot(a, c), np.dot(b, c)])


            alpha = h[0] / (A[0][0] + abs(A[1][0]))
            beta = h[1]  / (abs(A[0][1]) + A[1][1])

            n = alpha * a + beta * b - c

            gamma = np.linalg.norm(n)

            return str(alpha), str(beta), str(gamma)

    try:

        #
        # Check if all or any of the 3 vectors is zero
        
        if sum(c) != 0 and sum(a) == 0 and sum(b) == 0:
            return '1','1',str(dis.euclidean([0]*len(c),c))
        elif sum(c) == 0 and sum(a) == 0 and sum(b) == 0:
            return '1','1','0'
        elif sum(c) == 0 and sum(a) == 0:
            return '1','0','0'
        elif sum(c) == 0 and sum(b) == 0:
            return '0','1','0'

        else:
            
            
            # Matrix equation
            A = np.array([[np.dot(a, a), np.dot(a, b)], [np.dot(a, b), np.dot(b, b)]])

            h = np.array([np.dot(a, c), np.dot(b, c)])

            
            if A[0][0]==0 and h[0] ==0: #one vector zero, so combination can be only 1dim

                beta = h[1]/A[1][1]
                n = beta * b -c
                gamma = np.linalg.norm(n)

                if  (len(list(b)) - list(b).count(0) > 2) or np.linalg.norm(b) > 0.5:
                    return '1.0',str(beta),str(gamma)
                else:
                    return '1', '1', str(dis.euclidean([0] * len(c), c))
            elif A[1][1]==0 and h[1] ==0:
                alpha = h[0]/A[0][0]
                n = alpha * a -c
                gamma = np.linalg.norm(n)


                if len(list(a)) - list(a).count(0) > 2 or np.linalg.norm(a) > 0.5:
                    return str(alpha),'1',str(gamma)
                else:
                    return '1', '1', str(dis.euclidean([0] * len(c), c))
            elif h[0] == 0 and h[1] == 0:
                gamma = np.linalg.norm(c)
                return '0.0','0.0',str(gamma)
            elif A[0][0] != 0 and h[0] == 0 and h[1] == 0:
                gamma = np.linalg.norm(c)
                return '0.0','1.0',str(gamma)
            elif A[1][1] != 0 and h[0] == 0 and h[1] == 0:
                gamma = np.linalg.norm(c)
                return '1.0','0',str(gamma)

            
            ##########################
            # The ideal normal case with 3 vectors in correct angle and not being zero
            ##########################
            
            p = np.linalg.solve(A, h)
            # orthogonal vector
            n = p[0] * a + p[1] * b - c

            distance = np.linalg.norm(n)
            # check
            # print('dot product of a and c: %.4f' %(np.dot(a,n)))
            # print('dot product of b and c: %.4f' %(np.dot(b,n)))
            # print('distance: %.3f' %(distance))
            return str(p[0]), str(p[1]), str(distance)
    except:
        return 'Error', 'Error', 'Error'

### Calculate possible interactions for all pairs

In [ ]:
#BLISS RESULT FILE
fp_out = open('../results/Calculate_Interactions/Bliss_Scores_Cytotoxicity.csv','w')
fp_out.write('Drug1,Drug2,Batch,Plate,Well,S1,S2,C,Expected,Difference\n')

#MC RESULT FILE - Menche-Caldera Score
fp_out2 = open('../results/Calculate_Interactions/MC_Scores.csv','w')
fp_out2.write('Drug1,Drug2,Batch,Plate,Well,S1_Mahalanobis,S1_MPValue,S1_Norm,S2_Mahalanobis,S2_MPValue,S2_Norm,Combi_Mahalanobis,Combi_MPValue,Combi_Norm_Norm,Alpha,Beta,Gamma,Alpha_Zero,Beta_Zero,Gamma_Zero,Mahalanobis_Distance_To_NI,MPValue_To_NI\n')


#Results of the combination significances
Combination_Drug_Significance = {}

#Go through all pairs
for cloud1 in All_CLOUDs:
    for cloud2 in All_CLOUDs:
        # A|B = B|A
        if cloud1 > cloud2:

            
            #Get the combination well
            combination_well =  [x for x in well_cell_count if cloud1+','+cloud2 in x or cloud2+','+cloud1 in x]
            
            #This should never happen
            if len(combination_well) == 0:
                print 'Problem!!!'
            
            #Extract the plate and well information
            plate = combination_well[0].split(',')[2]
            well = combination_well[0].split(',')[3]
                         
            #Extract the batch information from the plate number
            if int(plate) < 1315065:
                b = 'Batch1'
            else:
                b= 'Batch2'
            
            #Check if either for the two drugs are in the drugs_to_remove list ==> all this interactions must be discarded (e.g. because the drug oxidized over course of screen)
            
            #Get the single replicates
            single1_replicate_wells = [x for x in drug_perturbation_vectors[b] if cloud1+',DMSO' in x]
            single2_replicate_wells = [x for x in drug_perturbation_vectors[b] if cloud2+',DMSO' in x]

            #Get the Combination well cell count, and the transfer status
            combi_Number =  well_cell_count[combination_well[0]]['Number']
            combi_Worked =  well_cell_count[combination_well[0]]['Worked']
            
            #Get the singles cell counts
            s1_cellCount = [well_cell_count[x]['Number'] for x in single1_replicate_wells if well_cell_count[x]['Worked'] == 'TRUE']
            s2_cellCount = [well_cell_count[x]['Number'] for x in single2_replicate_wells if well_cell_count[x]['Worked'] == 'TRUE']
            
            
            #If there was a problem with the drug transfer ===> all this interactions must be discarded (i.e. because no drug was transfered in this well)
            if combi_Worked == 'FALSE':
                fp_out.write(cloud1+','+cloud2+','+str(b)+','+plate+','+well+','+','.join(['CombinationTransferProblem']*5)+'\n')
                fp_out2.write(cloud1+','+cloud2+','+str(b)+','+plate+','+well+','+','.join(['CombinationTransferProblem']*17)+'\n')

            else:
                
                '''
                Perform CellCount Analysis:
                Calculate Bliss Score: E (Expected)= s1 + s2 - s1*s2; S (Score) = c - E 
                The actual score is the combination cytotoxicity minus what is expected.
                '''
               
                #Remove obvious outliers
                s1_cellCount_check = reject_outliers_2(s1_cellCount)
                s1_cellCount = [s1_cellCount[x] for x in range(0,len(s1_cellCount)) if s1_cellCount_check[x]]
                s2_cellCount_check = reject_outliers_2(s2_cellCount)
                s2_cellCount = [s2_cellCount[x] for x in range(0,len(s2_cellCount)) if s2_cellCount_check[x]]

                #Calculate the s1, s2 and c cytotoxicity; If the drug wells have even more cells than the DMSO control cytotoxicity is set to 0
                #Drug1
                s1_cytotoxicity = (DMSO_CellCount[b] -  np.mean(s1_cellCount))/DMSO_CellCount[b]
                #print s1_cytotoxicity
                if s1_cytotoxicity < 0:
                    s1_cytotoxicity = 0
                    
                #Drug2
                s2_cytotoxicity = (DMSO_CellCount[b] -  np.mean(s2_cellCount))/DMSO_CellCount[b]
                if s2_cytotoxicity < 0:
                    s2_cytotoxicity = 0

                #Combination
                comb_cytotoxicity = (DMSO_CellCount[b] -  combi_Number)/DMSO_CellCount[b]
                if comb_cytotoxicity < 0:
                    comb_cytotoxicity = 0
                
                               
                #Calculate the expected value (E); if e.g. both drug kill all cells the sum should also be only 100% and not 200%
                Bliss_expected = s1_cytotoxicity + s2_cytotoxicity - s1_cytotoxicity*s2_cytotoxicity
                if Bliss_expected > 1:
                    Bliss_expected = 1
                                     
                # Calculate the actual score by substracting expected value from the combination cytotoxicity.
                # POSITIVE values indicate cytotoxic syngergy while NEGATIVE values indicate cytotoxic antagony
                Bliss_Difference = comb_cytotoxicity - Bliss_expected
                
                #Write results to output
                fp_out.write(cloud1+','+cloud2+','+str(b)+','+plate+','+well+','+str(s1_cytotoxicity)+','+str(s2_cytotoxicity)+','+str(comb_cytotoxicity)+','+str(Bliss_expected)+','+str(Bliss_Difference)+'\n')

                #Check if one of the singles is a cytotoxic drug (i.e. has too few cells in the well) or the combination is cytotoxic
                if cloud1 in killing_drugs[b]:
                    fp_out2.write(cloud1+','+cloud2+','+str(b)+','+plate+','+well+','+','.join(['CytotoxicDrug1']*17)+'\n')
                    continue
                if cloud2 in killing_drugs[b]:
                    fp_out2.write(cloud1+','+cloud2+','+str(b)+','+plate+','+well+','+','.join(['CytotoxicDrug2']*17)+'\n')
                    continue
                if combi_Number < cutoff_min_cells:
                    fp_out2.write(cloud1+','+cloud2+','+str(b)+','+plate+','+well+','+','.join(['CytotoxicCombination']*17)+'\n')
                    continue
                
                '''
                MORPHOLOGY - MC SCORE CALCULATION
                '''
                    
                #################
                # FROM HERE actual morphology
                # 1.) check if the two singles and/or combination are significant morphological changers
                # 2.) depending on that calculate interactions
                ###########
                
                
                #Check if combination feature vector exist (if not there might have been a problem with the well; e.g. filtered for bad quality)
                if drug_perturbation_vectors[b].has_key(combination_well[0]) == False:
                    fp_out2.write(cloud1+','+cloud2+','+str(b)+','+plate+','+well+','+','.join(['ProblemWithCombinationWell']*17)+'\n')
                    continue
                
                #Get the combination vector
                combi_Vector =  drug_perturbation_vectors[b][combination_well[0]]
                
                
                # since only 1 replicate for combination k = only combi_Vector
                k = [combi_Vector]
                # number treatment = 1
                numberTreatment = len(k)
                # add ALL DMSO controls over the whole batch found  > 2k dmso wells
                k.extend(dmso[b])


                #Calculate the mahalanobis distance as well as empirical pValue (later not used in final paper)
                combination_distance_calc, combination_mp, pca1, pca2 = Calculate_MahalanobisDistance(k,numberTreatment,usePreComputedRandomDistribution=True,batch=b)
            
                #Calculate the vector length (=norm)
                Combi_VectorNorm = np.linalg.norm(combi_Vector)
            
                #Get the significance of drug1 and drug2
                s1_Mahalanobis = Single_Drug_Significance[cloud1][b]['Mahalanobis_Distance'] 
                s1_MValue = Single_Drug_Significance[cloud1][b]['MP_Value'] 
                s2_Mahalanobis = Single_Drug_Significance[cloud2][b]['Mahalanobis_Distance'] 
                s2_MValue = Single_Drug_Significance[cloud2][b]['MP_Value'] 
            
                #
                # Load the single vectors and calculate the MEAN vector for the respective replicates
                #
                
                ########
                # SINGLE 1
                #check if drug1 is significant (single1)
                s1_significant = False
                if s1_Mahalanobis > 7:
                    s1_significant = True
                s1_Vectors = drug_vectors_WithoutOutliers[b][cloud1].values()     

                #Create a numpy array
                val =  np.array(s1_Vectors)

                #Transform into mean vector
                s1_MeanVector = []
                for i in range(0,numfeatures):
                    s1_MeanVector.append(np.mean(val[:,i]))

                #get s1 vector length (=norm)
                s1_Mean_VectorNorm = np.linalg.norm(s1_MeanVector)

                ########
                # SINGLE 2
                #check if drug2 is significant (single2)
                s2_significant = False
                if  s2_Mahalanobis > 7:
                    s2_significant = True
                s2_Vectors = drug_vectors_WithoutOutliers[b][cloud2].values()    

                #Create a numpy array
                val =  np.array(s2_Vectors)

                #Transform into mean vector
                s2_MeanVector = []
                for i in range(0,numfeatures):
                    s2_MeanVector.append(np.mean(val[:,i]))

                #get s2 vector length (=norm)
                s2_Mean_VectorNorm = np.linalg.norm(s2_MeanVector)

                ########
                # Combination
                #check if combination is significant; no need to create a mean vector as only one replicate
                comb_significant = False
                if combination_distance_calc > 7:
                    comb_significant = True

                #Only if at least one of the three players is significantly perturbed
                #if no_interaction == False: 

                #Transform the three lists into numpy arrays
                s1_MeanVector_toCalculate = np.array(s1_MeanVector)
                s2_MeanVector_toCalculate = np.array(s2_MeanVector)
                comb_MeanVector_toCalculate = np.array(combi_Vector)              

                #Calculate alpha/beta/gamma
                alpha,beta,gamma = calculate_vector_math_v2(s1_MeanVector_toCalculate,s2_MeanVector_toCalculate,comb_MeanVector_toCalculate)


                '''
                Additionally one can also set non significant vector to the NULL vectors (then the math is forced to only use significant ones)
                '''

                if s1_significant == False:
                    s1_MeanVector_toCalculate = np.array([0]*numfeatures)                                
                if s2_significant == False:
                    s2_MeanVector_toCalculate = np.array([0]*numfeatures)                      
                if comb_significant == False:
                    comb_MeanVector_toCalculate = np.array([0]*numfeatures)

                
                #Calculate alpha_0/beta_0/gamma_0 (not used in final version)
                alpha_0,beta_0,gamma_0 = calculate_vector_math_v2(s1_MeanVector_toCalculate,s2_MeanVector_toCalculate,comb_MeanVector_toCalculate)


                '''
                CHECK IF THE INTERACTION IS SIGNIFICANT
                1. Calculate all possible vector sums of the single vectors
                2. Add intracellular plate variability by adding DMSO vectors
                3. Calculate the pValue and the Mahalanobis distance of the interaction to this 'possible vector sums'
                '''
                
                
                #Get DMSO wells (of this plate) after outlier detection (these serve ONLY as an ADDITIONAL cell fluctation that is added to all NI (vector sums))
                DMSO_Wells = dmso_wells_Outlier_Removed[b][plate]

                #Calculate all possible NIs and add the DMSO cell fluctuation to them (randomly choose 10 DMSO)
                possible_results = []
                for s1 in s1_Vectors:
                    tmp = []
                    for s2 in s2_Vectors:
                        vector_sum = np.array(s1) + np.array(s2)
                        possible_results.append(vector_sum)
                        #random_DMSO_Wells = random.sample(DMSO_Wells,25)
                        for DMSO_well in DMSO_Wells:
                            possible_results.append(vector_sum+np.array(DMSO_well))

                k = list(possible_results)
                k.insert(0,combi_Vector)
                #Here 500 random tries is already enough since only ca. 250 datapoints in 'possible' results
                combination_distance_calc_toNI, combination_mp_toNI, _, _ = Calculate_MahalanobisDistance(k,numberTreatment,number_of_RandomTries = 10000, usePreComputedRandomDistribution = False)


                #Make a PCA plot if it is a significant interaction
                if combination_distance_calc_toNI > 3:

                    #number of all points
                    number_vectorsums = len(k)

                    #colorize combination
                    color = ['#b2182b']*numberTreatment
                    for i in range(1,len(k)):
                        color.append('blue')

                    #add DMSO_Wells (for that plate); color = grey
                    k.extend(DMSO_Wells)
                    for i in range(0,len(DMSO_Wells)):
                        color.append('grey')

                    #add and color vector1
                    k.extend(s1_Vectors)
                    for i in range(0,len(s1_Vectors)):
                        color.append('#e08214')
                    
                    #add and color vector2
                    k.extend(s2_Vectors)
                    for i in range(0,len(s2_Vectors)):
                        color.append('#5aae61')

                    #Perform PCA dimension reduction
                    pca = PCA(n_components=2)
                    pca_vals = pca.fit_transform(k)

                    #add labels
                    if s1_significant:
                        cloud1_name = cloud1 +'(+)'
                    else:
                        cloud1_name = cloud1 +'(-)'

                    if s2_significant:
                        cloud2_name = cloud2 +'(+)'
                    else:
                        cloud2_name = cloud2 +'(-)'


                    #Create Legend
                    legend_elements = [Patch(facecolor='#b2182b', lw=1,
                                         label='Combination\nMahalanobis Distance: %.2f\n mp = %.2e' % (combination_distance_calc_toNI, combination_mp_toNI)),
                                       Patch(facecolor='blue', 
                                         label='Vector_Sums'),
                                       Patch(facecolor='grey', 
                                         label='DMSO'),
                                       Patch(facecolor='#e08214', 
                                         label=cloud1_name),
                                       Patch(facecolor='#5aae61', 
                                         label=cloud2_name)]

                    plt.figure(figsize=(5,5))
                    #Plot possible vector sums
                    sns.kdeplot(pca_vals[:, 0][1:number_vectorsums], pca_vals[:, 1][1:number_vectorsums], n_levels=8,cmap="Blues",alpha=0.6,gridsize = 100,shade=True, shade_lowest=False,  bw='scott', kernel='gau' )
                    #Plot DMSO, Single1 and Single2
                    plt.scatter(pca_vals[:, 0][number_vectorsums:], pca_vals[:, 1][number_vectorsums:], c=color[number_vectorsums:], alpha=0.6)
                    #Plot Combination
                    plt.scatter(pca_vals[:, 0][0], pca_vals[:, 1][0], c=color[0], alpha=1)
                    #Plot Legend
                    plt.legend(handles=legend_elements, loc='best', prop={'size': 7})
                    plt.xlim([min(pca_vals[:, 0]) - 0.1, max(pca_vals[:, 0])+0.1])
                    plt.ylim([min(pca_vals[:, 1]) - 0.1, max(pca_vals[:, 1])+0.1])
                    plt.xlabel('PC1 [%.2f]' %(pca.explained_variance_ratio_[0]))
                    plt.ylabel('PC2 [%.2f]' %(pca.explained_variance_ratio_[1]))
                    plt.savefig('../results/Calculate_Interactions/Combinations/'+cloud1+'_'+cloud2+'.png')
                    plt.close()

                #Write output
                fp_out2.write(cloud1+','+cloud2+','+str(b)+','+plate+','+well+','+str(s1_Mahalanobis)+','+str(s1_MValue)+','+str(s1_Mean_VectorNorm) +','+str(s2_Mahalanobis)+','+str(s2_MValue)+','+str(s2_Mean_VectorNorm)+','+str(combination_distance_calc)+','+str(combination_mp)+','+str(Combi_VectorNorm)+','+str(alpha)+','+str(beta)+','+str(gamma)+','+str(alpha_0)+','+str(beta_0)+','+str(gamma_0)+','+str(combination_distance_calc_toNI)+','+str(combination_mp_toNI) +'\n')
#Close file pointers                       
fp_out.close()
fp_out2.close()

print 'Created all interactions'

### Repair interaction files
In case something has to be added or removed

In [ ]:
from os import listdir
from os.path import isfile, join

mypath = '../results/Calculate_Interactions/MC_Scores/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

for file in onlyfiles:
    print file
    fp_out = open('../results/Calculate_Interactions/MC_ScoresNew/'+file,'w')
    
    fp = open(mypath+file)

    fp_out.write(fp.readline())
    for line in fp:
        tmp = line.strip().split(',')
        
        cloud1 = tmp[0]
        cloud2 = tmp[1]
        b = tmp[2]
        plate = tmp[3]
        well = tmp[4]
        
        combination_well =  [x for x in well_cell_count if cloud1+','+cloud2 in x or cloud2+','+cloud1 in x]
        combi_Number =  well_cell_count[combination_well[0]]['Number']
        
        if combi_Number < cutoff_min_cells:
             fp_out.write(cloud1+','+cloud2+','+str(b)+','+plate+','+well+','+','.join(['CytotoxicDrug']*19)+'\n')
        else:
            fp_out.write(line)
    fp.close()
    fp_out.close()
        
        

### Combine all files
This is only important if a computer cluster or any other tool is used to calculate the individual results files

In [ ]:
print 'Make Summary File'
print 'MC_Scores:'
MC_Score_Files = [f for f in os.listdir('../results/Calculate_Interactions/MC_Scores/') if os.path.isfile(os.path.join('../results/Calculate_Interactions/MC_Scores/', f))]
MC_Score_Files.sort()
fp_out = open('../results/Calculate_Interactions/All_MC_Scores.csv','w')
fp_out.write('Drug1,Drug2,Batch,Plate,Well,S1_Mahalanobis,S1_MPValue,S1_Norm,S2_Mahalanobis,S2_MPValue,S2_Norm,Combi_Mahalanobis,Combi_MPValue,Combi_Norm_Norm,Alpha,Beta,Gamma,Alpha_Zero,Beta_Zero,Gamma_Zero,DistanceFromOrigin,Mahalanobis_Distance_To_NI,MPValue_To_NI\n')
for file_name in MC_Score_Files:
    fp = open('../results/Calculate_Interactions/MC_Scores/'+file_name,'r')
    fp.next()
    for line in fp:
        fp_out.write(line)
    fp.close()
fp_out.close()
print 'Done'

print 'Bliss_Scores:'
Bliss_Score_Files = [f for f in os.listdir('../results/Calculate_Interactions/Bliss_Scores/') if os.path.isfile(os.path.join('../results/Calculate_Interactions/Bliss_Scores/', f))]
Bliss_Score_Files.sort()
fp_out = open('../results/Calculate_Interactions/All_Bliss_Scores.csv','w')
fp_out.write('Drug1,Drug2,Batch,Plate,Well,S1,S2,C,Expected,Difference\n')
for file_name in Bliss_Score_Files:
    fp = open('../results/Calculate_Interactions/Bliss_Scores/'+file_name,'r')
    fp.next()
    for line in fp:
        fp_out.write(line)
    fp.close()
fp_out.close()
print 'Done'